In [ ]:
# load new Scer haploid and diploid data, and check for depletions of events in essential genes in haploid
#
# essentiality list from https://www.yeastgenome.org/observable/APO:0000112


In [ ]:
# import modules
import pandas as pd
import pybedtools
import sys
import subprocess
from io import StringIO
from statistics import mean
import scipy.stats as stats
import pprint
#from gff3 import Gff3
#from BCBio.GFF import GFFExaminer
#import gffutils


In [9]:
# load data
PROJDIR = '/Volumes/CareyLab/Projects/2019__MicroHomologyMediatedIndels__XiangweHe_ZhejiangU/'
PROCDATA = PROJDIR + 'Sarah/new_MH_project_Spombe_Scer/ProcessedData_Spombe_Scer/'
PROCDATA = PROJDIR + 'DataFromCluster/NovaSeq6000_merged_counts.tsv/'
SGDDIR = '/Volumes/CareyLab/ExternalData/SGD/'
ScerEssential = pd.read_csv( SGDDIR + 'inviable_annotations.txt' , sep='\t')

SpomEssential = '/Volumes/CareyLab/ExternalData/PomBase/essential_genes.txt'
SpomEssential = pd.read_csv(SpomEssential , sep='\t')
SpomEssential = SpomEssential['systematic_ID']
#gff = Gff3()
#ScerGenes = gff.parse( '/Users/lcarey/Downloads/test.gff')
#ScerGenes = gff.parse( SGDDIR + 'saccharomyces_cerevisiae.gff')
#ScerGenes = gffutils.example_filename('/Users/lcarey/Downloads/test.gff')
#ScerGenes = pd.read_csv(SGDDIR + 'saccharomyces_cerevisiae_no_seq.gff3' , sep='\t' , comment='#' , header=0 , low_memory=False \
#           , names = ['chr','source','feature_type','feature_start','featured_end','score','strand','phase','attributes'])
#B5 = pd.read_csv( PROCDATA + 'B5_union.q30_f0x2.counts.tsv' , sep='\t' , comment='#' , header=0 , low_memory=False \
#           , names = ['chr','s1','e1','s2','e2','NDup','NCol'])
#B5['name'] = 'B5'


#ScerGenes = pybedtools.BedTool.from_dataframe(ScerGenes)

In [ ]:
# B5  LD388 exp   "diploid S. pombe, mid-log phase"   1   
# B6  DY1778 exp  "haploid S. pombe, mid-log phase"   1   
# B7  LD388 sat   "diploid S. pombe, saturated culture"   1   
# B8  DY1778 sat  "haploid S. pombe, saturated culture"   1   


In [12]:
# Intersect each .counts.tsv file with the gff with gene names
colnames = ['chr','s1','e2','NDup','NCol','GENE']
colnames = ['NDup','NCol','GENE']

GFF = '/Users/lcarey/CareyLab/ExternalData/PomBase/Schizosaccharomyces_pombe_all_chromosomes_only_geneIDs.gff3'
C2 = ' | bedtools intersect  -wb -f 1 -a - -b ' + GFF + ' | cut -f 1-5,14 ' # with chr position
C2 = ' | bedtools intersect  -wb -f 1 -a - -b ' + GFF + ' | cut -f 4,5,14 ' # bare minimum

TSV = PROCDATA + 'B6.q30_f0x2.counts.tsv'
CMD = '/usr/bin/cut -f 1,2,5,6,7  ' + TSV + C2 
a = subprocess.Popen( CMD , stdout=subprocess.PIPE , shell=True )
b = StringIO( a.communicate()[0].decode('utf-8'))
df6 = pd.read_csv( b , sep='\t' , names = colnames)

TSV = PROCDATA + 'B5.q30_f0x2.counts.tsv'
CMD = '/usr/bin/cut -f 1,2,5,6,7  ' + TSV + C2 
a = subprocess.Popen( CMD , stdout=subprocess.PIPE , shell=True )
b = StringIO( a.communicate()[0].decode('utf-8'))
df5 = pd.read_csv( b , sep='\t' , names = colnames)

TSV = PROCDATA + 'B7.q30_f0x2.counts.tsv'
CMD = '/usr/bin/cut -f 1,2,5,6,7  ' + TSV + C2 
a = subprocess.Popen( CMD , stdout=subprocess.PIPE , shell=True )
b = StringIO( a.communicate()[0].decode('utf-8'))
df7 = pd.read_csv( b , sep='\t' , names = colnames)

TSV = PROCDATA + 'B8.q30_f0x2.counts.tsv'
CMD = '/usr/bin/cut -f 1,2,5,6,7  ' + TSV + C2 
a = subprocess.Popen( CMD , stdout=subprocess.PIPE , shell=True )
b = StringIO( a.communicate()[0].decode('utf-8'))
df8 = pd.read_csv( b , sep='\t' , names = colnames)

In [13]:
idx5 = df5.GENE.isin(SpomEssential)
idx6 = df6.GENE.isin(SpomEssential)
idx7 = df5.GENE.isin(SpomEssential)
idx8 = df6.GENE.isin(SpomEssential)
x = [ [ sum(df6.NDup[idx6]), sum(df6.NDup[~idx6]) ] , [ sum(df5.NDup[idx5]), sum(df5.NDup[~idx5]) ] ]
pprint.pprint(x)
oddratio , pvalue = stats.fisher_exact( x )
print(f"OR = {oddratio} , p = {pvalue}")

x = [ [ sum(df8.NDup[idx8]), sum(df8.NDup[~idx8]) ] , [ sum(df7.NDup[idx7]), sum(df7.NDup[~idx7]) ] ]
pprint.pprint(x)
oddratio , pvalue = stats.fisher_exact( x )
print(f"OR = {oddratio} , p = {pvalue}")

[[0, 2], [0, 5]]
OR = nan , p = 1.0
[[0, 2], [0, 5]]
OR = nan , p = 1.0


20